In [ ]:
import param
import numpy as np
import holoviews as hv
hv.notebook_extension('bokeh', 'matplotlib')

In previous notebooks we discovered how the [DynamicMap](Dynamic_Map.ipynb) class allows us to declare objects in a lazy way to enable exploratory analysis of large parameter spaces. In the [Streams](Streams.ipynb) tutorial we learned how to interactively push updates to existing plots by declaring Streams on a DynamicMap. In this tutorial we will extend the idea to so called **linked** Streams, which allow declaring complex interactions by exposing events generated by interacting with a plot. Being able to succinctly capture interactions with a plot and defining a simple Python based callback to define the interactivity allows generating much richer visualizations, revealing more detail about your data.

Some of the many possibilities this opens up include:

* Dynamically aggregating datasets of billions of datapoints depending on the plot axis ranges using the [datashader](https://anaconda.org/jbednar/holoviews_datashader/notebook) library.
* Responding to Tap and DoubleTap events to reveal more information in subplots.
* Computing statistics in response to selections applied with box- and lasso-select tools.

Currently only the bokeh backend for HoloViews supports linked streams but in theory any backend can define callback which fire when a user zooms or pans or interacts with a plot.

<center><div class="alert alert-info" role="alert">To use and visualize <b>DynamicMap</b> or <b>Stream</b> objects you need to be running a live Jupyter server.<br>This tutorial assumes that it will be run in a live notebook environment.<br>
When viewed statically, DynamicMaps will only show the first available Element,<br></center></div>

## Available Linked Streams

There a huge number of ways one might want to interact with a plot. The HoloViews streams module aims to offer many of the most common interactions you might want to specify while making it easy to extend it with custom linked Streams. 

We can see the full list of linked Stream types by iterating over the descendents of the ``LinkedStream`` baseclass.

In [ ]:
from holoviews import streams
sorted([str(s) for s in param.descendents(streams.LinkedStream)])

As you can see, most of these events are about specific interactions with a plot such as the current axis ranges (``Range[X][Y]``), the mouse pointer position (``Pointer[X][Y]``), click or tap positions (``Tap``, ``DoubleTap``). Additionally there a streams to access plotting selections made using box- and lasso-select tools (``Selection1D``), the plot size (``PlotSize``) and the ``Bounds`` of a selection. 

Each of these linked Stream types has a corresponding backend specific ``Callback``, which defines which plot attributes or events to link the stream to and triggers events on the ``Stream`` in response to changes on the plot. Defining custom ``Stream`` and ``Callback`` types will be covered in the [Stream Callback Tutorial](Stream_Callbacks.ipynb).

## Linking streams to plots

In the [Streams](./Streams.ipynb) tutorial we discovered that streams have ``subscribers``, which allow defining user defined callbacks on events, but also allow plots to subscribe to Stream updates. Linked streams add another concept on top of ``subscribers``, the ``source`` of a Stream.

The source of a linked stream defines which plot element to receive events from. Any plot containing the ``source`` object will be linked to the stream and send events in response to changes.

Let's start with a simple example. We will declare one of the linked Streams from above, the ``PointerXY`` stream. This stream sends the current mouse position in plot axes coordinates, which may be continuous or categorical. The first thing to note is that since we have passed no ``source`` the attribute is ``None``.

In [ ]:
pointer = streams.PointerXY()
print pointer.source

Before continuing we can check which keywords the Stream makes available to a user by printing it or looking at its contents:

In [ ]:
print('The %s stream contains: %r' % (pointer, pointer.contents))

#### Automatic linking

A Stream is automatically linked to the first DynamicMap we attach the Stream to, which we can confirm by inspecting the stream's ``source`` attribute:

In [ ]:
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])
print pointer.source is pointer_dmap

The DynamicMap we defined above simply defines returns a Points object containing the current x, y coordinate of our ``PointerXY`` stream and is now linked to any plot this object is displayed in:

In [ ]:
pointer_dmap(style={"Points": dict(size=10)})

If you hover over the plot canvas above you can see how the point tracks the current mouse position. We can also inspect the last cursor position on the stream:

In [ ]:
pointer

#### Explicit linking

In the example above we just let took advantage of the automatic linking to the object we attached the stream to. If we want to link the Stream to a different object we can pass an explicit source. Here we will create a 2D Image of sine gratings,  declare that object as the ``source`` of the ``PointerXY`` stream, and then use the pointer stream to define a single point showing the cursor position on the image.

In [ ]:
xvals = np.linspace(0,4,202)
ys,xs = np.meshgrid(xvals, -xvals[::-1])
img = hv.Image(np.sin(((ys)**3)*xs))

pointer = streams.PointerXY(source=img)
pointer_dmap = hv.DynamicMap(lambda x, y: hv.Points([(x, y)]), streams=[pointer])

Now if we display the ``Image`` and the ``DynamicMap`` together in a ``Layout``, the plot on the right will track the cursor position on image plot to the left, which we designated as the source:

In [ ]:
img + pointer_dmap(style={"Points": dict(size=10)})

This will even work across different cells. If we add the stream to another object and display it, the new plot will be linked to the cursor position on the image plot. 

We will use the pointer x and y position to take cross-sections of the image at the hover location using the ``Image.sample`` method. We also ensure that the position does not go out of bounds.

In [ ]:
%%opts Curve {+framewise}
hv.DynamicMap(lambda x, y: img.sample(y=y if -.5<y<.5 else 0), streams=[pointer]) +\
hv.DynamicMap(lambda x, y: img.sample(x=x if -.5<x<.5 else 0), streams=[pointer])

Now try hovering over the ``Image`` above again and watch the cross-sections update as you move the cursor.

#### Unlinking streams

As we already found out, linked streams are automatically linked to plots containing the ``source`` of the stream. However, particularly while testing or debugging a callback, it can be useful to disable the linking.

In this example we will declare a ``RangeX`` stream which is usually linked to the x-axis range of the plot. By setting ``linked=False``, we can disable the automatic linking, so we can update the stream manually. 

As a simple example we declare a sine curve and then slice the curve with the provided ``x_range``.

In [ ]:
xs = np.linspace(0, np.pi*4)
curve = hv.Curve((xs, np.sin(xs)))
rangex = streams.RangeX(linked=False, x_range=curve.range('x'))
hv.DynamicMap(lambda x_range: curve.select(x=x_range), streams=[rangex])

Since the stream is not linked it will not trigger events when you zoom and pan. Instead we will update the range manually by iterating over the x-values and progressively expanding the range. Observe how the curve animates when you run the loop:

In [ ]:
for x in xs:
    rangex.event(x_range=(0, x))

#### Unlinking objects

Sometimes we just want to display an object designated as a source without linking it to the stream. If the object is not a DynamicMap, like the Image we designated as a ``source`` above, we can make a copy of the object with the ``clone`` method. However if it is a DynamicMap we have to make a copy with the ``Dynamic`` utility and declare ``link_inputs=False``.

Here we will create a DynamicMap that draws a cross-hair at the cursor position:

In [ ]:
pointer = streams.PointerXY(x=0, y=0)
cross_dmap = hv.DynamicMap(lambda x, y: (hv.VLine(x) * hv.HLine(y)), streams=[pointer])

Now we will add two copies of the ``cross_dmap`` into a Layout but the subplot on the right will not ``link_inputs``. Try hovering over the two subplots and observe what happens:

In [ ]:
cross_dmap + hv.util.Dynamic(cross_dmap, link_inputs=False)

Notice how hovering over the left plot updates the crosshair position on both subplots, while hovering over the right subplot has no effect.

## Transient linked streams

In the basic [Streams](Streams.ipynb) tutorial we already discovered the idea of transient streams, which reset their values to ``None`` when the stream is not active. Transient streams are particularly useful when you are subscribed to multiple streams sending discrete events. A good example are the ``Tap`` and ``DoubleTap`` streams; while you sometimes just want to know the last tapped position, we can only tell the two events apart if their values are ``None`` when not active. 

We'll start by declaring a ``Tap`` and a ``DoubleTap`` stream as ``transient``. Since both streams supply 'x' and 'y' parameters, we will rename the ``DoubleTap`` parameters to 'x2' and 'y2'.

In [ ]:
tap = streams.Tap(transient=True)
double_tap = streams.DoubleTap(rename={'x': 'x2', 'y': 'y2'}, transient=True)

Next we define a list of taps we can append to and a function that performs that accumulates the tap and double tap coordinate along with the number of taps, returning a ``Points`` Element of the tap positions.

In [ ]:
taps = []

def record_taps(x, y, x2, y2):
    if x and y:
        taps.append((x, y, 1))
    elif x2 and y2:
        taps.append((x2, y2, 2))
    return hv.Points(taps, vdims=['Taps'])

Finally we can create a ``DynamicMap`` from our callback and attach the streams. We also apply some styling so the points are colored depending on the number of taps.

In [ ]:
%%opts Points [color_index='Taps' tools=['hover']] (size=10 cmap='Set1')
hv.DynamicMap(record_taps, streams=[tap, double_tap])

Now try single- and double-tapping within the plot area, each time you tap a new point is appended to the list and displayed. We can also inspect the list of taps directly:

In [ ]:
taps